### Añadimos SRC a la raíz del proyecto para poder importar el contenido

In [1]:
import sys
import os

# Obtener la ruta absoluta de la carpeta raíz (donde está src)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde notebooks/

# Agregar la carpeta src al path
sys.path.append(os.path.join(ROOT_DIR, "src"))

### Importamos los módulos necesarios

In [2]:
import pandas as pd
import time
from pokemon_api import get_pokemon_names, get_all_forms, get_pokemon_data, get_species_data, clean_pokemon_name
from data_processing import save_partial_data, filter_pokemon_variants
from config import PARTIAL_SAVE_PATH_DATA_COLLECTION, FINAL_SAVE_PATH_DATA_COLLECTION, FILTER_DATA_PATH, SORTED_FILTER_DATA_PATH, SORTED_FILTER_DATA_PATH_CLEANED, FINAL_DATASET_PATH

### Cargamos el dataset recolectado en 01_data_collection.ipynb

In [3]:
# Cargar el dataset original
df_pokemon = pd.read_csv(FINAL_SAVE_PATH_DATA_COLLECTION)

In [4]:
df_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1233 entries, 0 to 1232
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           1233 non-null   int64  
 1   name            1233 non-null   object 
 2   type            1231 non-null   object 
 3   height          1233 non-null   float64
 4   weight          1233 non-null   float64
 5   abilities       1231 non-null   object 
 6   hidden_ability  973 non-null    object 
 7   hp              1233 non-null   int64  
 8   attack          1233 non-null   int64  
 9   defense         1233 non-null   int64  
 10  sp_atk          1233 non-null   int64  
 11  sp_def          1233 non-null   int64  
 12  speed           1233 non-null   int64  
 13  legendary       1233 non-null   bool   
 14  mythical        1233 non-null   bool   
 15  generation      1233 non-null   object 
dtypes: bool(2), float64(2), int64(7), object(5)
memory usage: 137.4+ KB


In [5]:
print("\n🔎 Valores nulos en el dataset:")
print(df_pokemon.isnull().sum())


🔎 Valores nulos en el dataset:
index               0
name                0
type                2
height              0
weight              0
abilities           2
hidden_ability    260
hp                  0
attack              0
defense             0
sp_atk              0
sp_def              0
speed               0
legendary           0
mythical            0
generation          0
dtype: int64


In [6]:
print("\n📌 Pokémon únicos en el dataset:")
print(df_pokemon["name"].nunique())


📌 Pokémon únicos en el dataset:
1233


In [7]:
print("\n🔍 Duplicados en el dataset:")
print(df_pokemon.duplicated().sum())


🔍 Duplicados en el dataset:
0


### Eliminamos los Pokemon que tengan variante y tengan las mismas habilidades y estadísticas.

In [8]:
# Cargar el dataset original
df_pokemon = pd.read_csv(FINAL_SAVE_PATH_DATA_COLLECTION)

# Identificar Pokémon con formas alternativas
duplicated_pokemon = df_pokemon[df_pokemon.duplicated(subset=["index"], keep=False)]

# Columnas para comparación
stats_columns = ["hp", "attack", "defense", "sp_atk", "sp_def", "speed"]
ability_columns = ["abilities", "hidden_ability"]

# Lista para almacenar Pokémon filtrados
filtered_pokemon = []

# LOG: Pokémon que tienen variantes
print(f"🔎 Encontrados {duplicated_pokemon['index'].nunique()} Pokémon con formas alternativas.")

# Iterar por cada Pokémon con formas alternativas
for index, group in duplicated_pokemon.groupby("index"):
    base_form = group.iloc[0]  # Tomamos la primera forma como base

    # LOG: Mostrar qué Pokémon estamos analizando
    print(f"\n🔍 Analizando {base_form['name']} (ID: {index}) - {len(group)} formas encontradas.")

    # Filtrar variantes distintas en estadísticas o habilidades
    unique_forms = group.drop_duplicates(subset=stats_columns + ability_columns)

    if len(unique_forms) > 1:
        # LOG: Si hay diferencias en habilidades o estadísticas, conservar todas las variantes
        print(f"✅ Variantes diferentes detectadas para {base_form['name']}. Se mantienen todas.")
        filtered_pokemon.append(unique_forms)
    else:
        # LOG: Si todas las variantes son idénticas, solo conservar la base
        print(f"❌ Todas las variantes de {base_form['name']} son idénticas. Se mantiene solo la base.")
        filtered_pokemon.append(base_form.to_frame().T)

# Concatenar todas las filas filtradas
df_filtered = pd.concat(filtered_pokemon, ignore_index=True)

# Verificar si los Pokémon base están en `df_filtered`
print("\n🔎 Verificando Pokémon filtrados antes de fusionar...")
print(df_filtered[df_filtered["name"].isin(["venusaur", "charizard", "blastoise"])].head())

# LOG: Pokémon filtrados y cuántos se conservaron
print(f"\n📊 Pokémon con formas que se han conservado tras el filtrado: {df_filtered.shape[0]}")

# Agregar Pokémon base sin variantes para completar el dataset
df_final = pd.concat([
    df_pokemon[~df_pokemon["index"].isin(duplicated_pokemon["index"])],  # Pokémon sin variantes
    df_filtered  # Pokémon con formas pero filtradas correctamente
], ignore_index=True)

# Verificar si `df_final` contiene los Pokémon base antes de guardarlo
print("\n🔎 Verificando Pokémon base en `df_final` antes de guardar...")
print(df_final[df_final["name"].isin(["venusaur", "charizard", "blastoise"])].head())

# LOG: Recuento final
print(f"📊 Pokémon totales en el dataset final: {df_final.shape[0]}")

# Resetear el índice
df_final.reset_index(drop=True, inplace=True)

# Guardar el dataset limpio

df_final.to_csv(FILTER_DATA_PATH, index=False)

print(f"✅ Dataset filtrado guardado correctamente en '{FILTER_DATA_PATH}'.")

# Verificar contenido del archivo guardado
df_check = pd.read_csv(FILTER_DATA_PATH)
print("\n🔎 Verificando contenido del CSV guardado...")
print(df_check[df_check["name"].isin(["venusaur", "charizard", "blastoise"])].head())

🔎 Encontrados 158 Pokémon con formas alternativas.

🔍 Analizando venusaur (ID: 3) - 3 formas encontradas.
✅ Variantes diferentes detectadas para venusaur. Se mantienen todas.

🔍 Analizando charizard (ID: 6) - 4 formas encontradas.
✅ Variantes diferentes detectadas para charizard. Se mantienen todas.

🔍 Analizando blastoise (ID: 9) - 3 formas encontradas.
✅ Variantes diferentes detectadas para blastoise. Se mantienen todas.

🔍 Analizando butterfree (ID: 12) - 2 formas encontradas.
❌ Todas las variantes de butterfree son idénticas. Se mantiene solo la base.

🔍 Analizando beedrill (ID: 15) - 2 formas encontradas.
✅ Variantes diferentes detectadas para beedrill. Se mantienen todas.

🔍 Analizando pidgeot (ID: 18) - 2 formas encontradas.
✅ Variantes diferentes detectadas para pidgeot. Se mantienen todas.

🔍 Analizando rattata (ID: 19) - 2 formas encontradas.
✅ Variantes diferentes detectadas para rattata. Se mantienen todas.

🔍 Analizando raticate (ID: 20) - 3 formas encontradas.
✅ Variantes

### Ordenamos por Nº de Pokedex y Nombre base siempre primero

In [9]:
# Cargar el archivo CSV filtrado

df_final = pd.read_csv(FILTER_DATA_PATH)

# Crear una nueva columna para identificar la forma base (sin '-')
df_final["is_base"] = df_final["name"].apply(lambda x: "-" not in x)

# Ordenar por index y asegurarse de que la forma base quede en primer lugar
df_final = df_final.sort_values(by=["index", "is_base"], ascending=[True, False]).reset_index(drop=True)

# Eliminar la columna auxiliar "is_base" después de ordenar
df_final.drop(columns=["is_base"], inplace=True)

# Guardar nuevamente el CSV ordenado correctamente

df_final.to_csv(SORTED_FILTER_DATA_PATH, index=False)

print(f"✅ Dataset ordenado y limpio guardado correctamente en '{SORTED_FILTER_DATA_PATH}'.")

# Mostrar las primeras filas para verificar
df_final.head(20)

✅ Dataset ordenado y limpio guardado correctamente en '../data/data_filtered/pokemon_filtered_sorted_data.csv'.


,index,name,type,height,weight,abilities,hidden_ability,hp,attack,defense,sp_atk,sp_def,speed,legendary,mythical,generation
0,1,bulbasaur,grass / poison,0.7,6.9,overgrow,chlorophyll,45,49,49,65,65,45,False,False,generation-i
1,2,ivysaur,grass / poison,1.0,13.0,overgrow,chlorophyll,60,62,63,80,80,60,False,False,generation-i
2,3,venusaur,grass / poison,2.0,100.0,overgrow,chlorophyll,80,82,83,100,100,80,False,False,generation-i
3,3,venusaur-mega,grass / poison,2.4,155.5,thick-fat,NaN,80,100,123,122,120,80,False,False,generation-i
4,4,charmander,fire,0.6,8.5,blaze,solar-power,39,52,43,60,50,65,False,False,generation-i
5,5,charmeleon,fire,1.1,19.0,blaze,solar-power,58,64,58,80,65,80,False,False,generation-i
6,6,charizard,fire / flying,1.7,90.5,blaze,solar-power,78,84,78,109,85,100,False,False,generation-i
7,6,charizard-mega-x,fire / dragon,1.7,110.5,tough-claws,NaN,78,130,111,130,85,100,False,False,generation-i
8,6,charizard-mega-y,fire / flying,1.7,100.5,drought,NaN,78,104,78,159,115,100,False,False,generation-i
9,7,squirtle,water,0.5,9.0,torrent,rain-dish,44,48,65,50,64,43,False,False,generation-i


### Eliminar a frillish-female y jellicent-female, ya que dan errores

In [10]:

# Cargar el dataset en un DataFrame
df = pd.read_csv(SORTED_FILTER_DATA_PATH)

# Eliminar las filas con los nombres específicos
df = df[~df["name"].isin(["frillish-female", "jellicent-female"])]

# Guardar el dataset actualizado
df.to_csv(SORTED_FILTER_DATA_PATH, index=False)

print("✅ Filas eliminadas y dataset actualizado correctamente.")

✅ Filas eliminadas y dataset actualizado correctamente.


### Rellenar valores nulos de "hidden_ability" con valor "None" y crear columna "total_stats" con la suma de todas las estadísticas base

In [36]:
# Cargar el archivo CSV ordenado

df = pd.read_csv(SORTED_FILTER_DATA_PATH)

# Rellenar valores nulos en la columna "hidden_ability" con "None"
df["hidden_ability"].fillna("None ", inplace=True)

# Crear la nueva columna "total_stats" sumando todas las estadísticas base
stats_columns = ["hp", "attack", "defense", "sp_atk", "sp_def", "speed"]
df["total_stats"] = df[stats_columns].sum(axis=1)

columns_to_convert = ["index", "height", "weight", "hp", "attack", "defense", "sp_atk", "sp_def", "speed", "total_stats"]
df[columns_to_convert] = df[columns_to_convert].astype(int)

# Guardar el dataset actualizado
df.to_csv(SORTED_FILTER_DATA_PATH_CLEANED, index=False)

print(f"✅ Dataset actualizado y guardado en '{SORTED_FILTER_DATA_PATH_CLEANED}'.")

# Mostrar una muestra de los datos para verificar
display(df.head(20))


✅ Dataset actualizado y guardado en '../data/data_filtered/pokemon_filtered_sorted_data_cleaned.csv'.


C:\Users\furio\AppData\Local\Temp\ipykernel_19892\301948155.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["hidden_ability"].fillna("None ", inplace=True)


,index,name,type,height,weight,abilities,hidden_ability,hp,attack,defense,sp_atk,sp_def,speed,legendary,mythical,generation,total_stats
0,1,bulbasaur,grass / poison,0,6,overgrow,chlorophyll,45,49,49,65,65,45,False,False,generation-i,318
1,2,ivysaur,grass / poison,1,13,overgrow,chlorophyll,60,62,63,80,80,60,False,False,generation-i,405
2,3,venusaur,grass / poison,2,100,overgrow,chlorophyll,80,82,83,100,100,80,False,False,generation-i,525
3,3,venusaur-mega,grass / poison,2,155,thick-fat,None,80,100,123,122,120,80,False,False,generation-i,625
4,4,charmander,fire,0,8,blaze,solar-power,39,52,43,60,50,65,False,False,generation-i,309
5,5,charmeleon,fire,1,19,blaze,solar-power,58,64,58,80,65,80,False,False,generation-i,405
6,6,charizard,fire / flying,1,90,blaze,solar-power,78,84,78,109,85,100,False,False,generation-i,534
7,6,charizard-mega-x,fire / dragon,1,110,tough-claws,None,78,130,111,130,85,100,False,False,generation-i,634
8,6,charizard-mega-y,fire / flying,1,100,drought,None,78,104,78,159,115,100,False,False,generation-i,634
9,7,squirtle,water,0,9,torrent,rain-dish,44,48,65,50,64,43,False,False,generation-i,314


### Verificamos que no tenemos valores nulos

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           1151 non-null   int64 
 1   name            1151 non-null   object
 2   type            1151 non-null   object
 3   height          1151 non-null   int64 
 4   weight          1151 non-null   int64 
 5   abilities       1151 non-null   object
 6   hidden_ability  1151 non-null   object
 7   hp              1151 non-null   int64 
 8   attack          1151 non-null   int64 
 9   defense         1151 non-null   int64 
 10  sp_atk          1151 non-null   int64 
 11  sp_def          1151 non-null   int64 
 12  speed           1151 non-null   int64 
 13  legendary       1151 non-null   bool  
 14  mythical        1151 non-null   bool  
 15  generation      1151 non-null   object
 16  total_stats     1151 non-null   int64 
dtypes: bool(2), int64(10), object(5)
memory usage: 137.3

### Ya tenemos el dataset final, lo renombramos a pokemon_dataset_final.csv

In [38]:
import shutil

# Ruta del archivo original y nuevo nombre
old_path = SORTED_FILTER_DATA_PATH_CLEANED
new_path = FINAL_DATASET_PATH

# Copiar el archivo
shutil.copy(old_path, new_path)

print("✅ El archivo ha sido copiado correctamente como 'pokemon_dataset_final.csv'.")

✅ El archivo ha sido copiado correctamente como 'pokemon_dataset_final.csv'.


In [43]:
df = pd.read_csv(FINAL_DATASET_PATH)
df.head(12)

,index,name,type,height,weight,abilities,hidden_ability,hp,attack,defense,sp_atk,sp_def,speed,legendary,mythical,generation,total_stats
0,1,bulbasaur,grass / poison,0,6,overgrow,chlorophyll,45,49,49,65,65,45,False,False,generation-i,318
1,2,ivysaur,grass / poison,1,13,overgrow,chlorophyll,60,62,63,80,80,60,False,False,generation-i,405
2,3,venusaur,grass / poison,2,100,overgrow,chlorophyll,80,82,83,100,100,80,False,False,generation-i,525
3,3,venusaur-mega,grass / poison,2,155,thick-fat,None,80,100,123,122,120,80,False,False,generation-i,625
4,4,charmander,fire,0,8,blaze,solar-power,39,52,43,60,50,65,False,False,generation-i,309
5,5,charmeleon,fire,1,19,blaze,solar-power,58,64,58,80,65,80,False,False,generation-i,405
6,6,charizard,fire / flying,1,90,blaze,solar-power,78,84,78,109,85,100,False,False,generation-i,534
7,6,charizard-mega-x,fire / dragon,1,110,tough-claws,None,78,130,111,130,85,100,False,False,generation-i,634
8,6,charizard-mega-y,fire / flying,1,100,drought,None,78,104,78,159,115,100,False,False,generation-i,634
9,7,squirtle,water,0,9,torrent,rain-dish,44,48,65,50,64,43,False,False,generation-i,314


In [44]:
# Mostrar las filas donde 'hidden_ability' es nulo
df_null_hidden_ability = df[df["hidden_ability"].isna()]

# Mostrar el resultado en el notebook
print(df_null_hidden_ability)


Empty DataFrame
Columns: [index, name, type, height, weight, abilities, hidden_ability, hp, attack, defense, sp_atk, sp_def, speed, legendary, mythical, generation, total_stats]
Index: []
